this nb just scrapes the data. nothing else. doesn't parse, etc. i think we can parse later. 

sometimes code is too fast for browser driver. in such cases, try increasing sleep time.

In [1]:
from bs4 import BeautifulSoup as bs
from datetime import datetime
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
from enum import Enum
import re
import pandas as pd
 

In [3]:
DRIVER_PATH = 'chromedriver_win32/chromedriver.exe'
SEARCH_TERM = 'asked the' # can be blank
# PARLIAMENT_INDEX = -1 # legislative assembly = 1, 1st parliament = 2, etc etc. -1 to ignore this field.

In [4]:
def element_completely_viewable(driver, elem):
    elem_left_bound = elem.location.get('x')
    elem_top_bound = elem.location.get('y')
    elem_width = elem.size.get('width')
    elem_height = elem.size.get('height')
    elem_right_bound = elem_left_bound + elem_width
    elem_lower_bound = elem_top_bound + elem_height

    win_upper_bound = driver.execute_script('return window.pageYOffset')
    win_left_bound = driver.execute_script('return window.pageXOffset')
    win_width = driver.execute_script('return document.documentElement.clientWidth')
    win_height = driver.execute_script('return document.documentElement.clientHeight')
    win_right_bound = win_left_bound + win_width
    win_lower_bound = win_upper_bound + win_height

    return all((win_left_bound <= elem_left_bound,
                win_right_bound >= elem_right_bound,
                win_upper_bound <= elem_top_bound,
                win_lower_bound >= elem_lower_bound)
              )

def find_element(driver, elem):
    y_height = 0
    while not element_completely_viewable(driver, elem):
        driver.execute_script(f"window.scrollTo(0, {y_height})") 
        y_height += 300
        time.sleep(0.25)

def find_and_click(driver, elem):
    find_element(driver, elem)
    if element_completely_viewable(driver, elem):
        elem.click()
    else:
        print('finding and clicking agn')
        find_and_click(driver, elem)



In [5]:
driver = webdriver.Chrome(DRIVER_PATH) 
 
page_url = 'https://sprs.parl.gov.sg/search/home' 
driver.get(page_url) 
driver.maximize_window() 
driver.implicitly_wait(20) 
time.sleep(2) 
 
# Get search box and fill it up 

search = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[1]/div/div[1]/input')
search.send_keys(SEARCH_TERM) 

# exact phrase search

search_opt = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[2]/div/div[1]/select')
find_and_click(driver, search_opt)
exact_search_opt = driver.find_element(by=By.XPATH, value='//*[@id="divmpscreen2"]/div[2]/div[2]/div/div[1]/select/option[3]')
find_and_click(driver, exact_search_opt)
 
# Uncomment following two lines to only search in titles # 
#checkbox = driver.find_element_by_xpath('//*[@id="divmpscreen2"]/div[2]/div[1]/div/label/input')
#checkbox.click() 

# from 12th parliament to 14th.
for i in range(13, 16):
    session = driver.find_element(
        by=By.XPATH, 
        value=f'/html/body/app-root/app-search/div/div[2]/div[2]/div[1]/div/div[3]/select/option[{i}]')
    session.click() 
 
# advanced search - only look for oral and written answers to questions

adv_search_opt = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[3]/div/div[1]/h5/a')
find_and_click(driver, adv_search_opt)
section_selection = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[3]/div/div[2]/div/div/div/div/div[3]/div/select')
find_element(driver, section_selection)
for selection_option_index in [13, 19, 20]: # corresponds to answers to questions    
    option = driver.find_element(
        by=By.XPATH, 
        value=f'/html/body/app-root/app-search/div/div[3]/div/div[2]/div/div/div/div/div[3]/div/select/option[{selection_option_index}]')
    option.click()


# Find submit element and press enter

submit = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[3]/div/button[2]') 
submit.send_keys(Keys.RETURN) 

print('Search parameters submitted.') 

# Switch window and check for number of search results 

driver.switch_to.window(driver.window_handles[1])
driver.implicitly_wait(20) 
time.sleep(2) 

try:
    num_res_label = driver.find_element(by=By.CLASS_NAME, value='showingResults')
except: 
    print('Error when finding num_res_label')
    
num_res_text = num_res_label.get_attribute('innerHTML')
    
if 'No' in num_res_text:
    print('no results found') # but site is quite erratic, may take a few tries to get results (idk why)
else:
    num_results = int(num_res_text[num_res_text.rfind(' ')+1:])
    num_pages = math.ceil(num_results / 20)
    print(f'{num_results} results found spanning {num_pages} page(s)')

C:\Users\thisi\AppData\Local\Temp\ipykernel_3284\2364195373.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


Search parameters submitted.
4525 results found spanning 227 page(s)


In [6]:
docs_saved = 0

In [7]:
docs_to_skip = 1800 # should set to 0. use this if it crashes halfway.
docs_saved = 0
pages_seen = 0

while (True):
    driver.switch_to.window(driver.window_handles[1])
    
    if docs_to_skip < 20:

        elems = driver.find_elements(by=By.XPATH, value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/table/tbody')
        links = list(map(lambda elem: elem.find_elements(by=By.XPATH, value='.//tr[1]/td[2]/a')[0], elems))

        for link in links:
            if docs_to_skip > 0:
                docs_to_skip -= 1
                docs_saved += 1
                continue
                
            try:
                find_and_click(driver, link)

                # Switch to page with content and get URL name
                time.sleep(1)
                while len(driver.window_handles) < 3:
                    find_and_click(driver, link)
                    print(f'trying this link agn')
                    time.sleep(2)

                driver.switch_to.window(driver.window_handles[2])
                
                item_key = driver.current_url.split('/')[-1]
                item_key = item_key.replace('?', '_') # Replace ? because it would be an invalid filename 
                
                page_source = driver.page_source

                with open(f'scraped_content/{item_key}.html', 'w') as f:
                    f.write(page_source)
                
                docs_saved += 1
                if docs_saved % 20 == 0:
                    print(f'Saved {docs_saved} / {num_results} docs at {datetime.now().strftime("%H%M")}')

                # Close tab         
                driver.close()
                driver.switch_to.window(driver.window_handles[1])
                time.sleep(1)
            except Exception as e:
                print('exception' + str(e))
                #import pdb
                #pdb.set_trace()
                
                docs_saved += 1
                driver.close()
                driver.switch_to.window(driver.window_handles[1])
                time.sleep(1)
                continue
    else:
        docs_saved += 20
        docs_to_skip -= 20
        print('skipping')
        
    if docs_saved == num_results:
        break
    elif docs_saved == 20: # if we're on the first results page then there is no previous page button
        next_page = driver.find_element(
            by=By.XPATH, 
            value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/div[3]/section/ul/li[1]/a')
    else: # if we're not on the first results page then there is a previous page button
        next_page = driver.find_element(
            by=By.XPATH, 
            value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/div[3]/section/ul/li[3]/a')
    
    find_and_click(driver, next_page)
    pages_seen += 1
    print(f'Seen {pages_seen} / {num_pages} pages')

    # Sleep momentarily because next page takes a while to load     
    time.sleep(2) 


skipping
Seen 1 / 227 pages
skipping
Seen 2 / 227 pages
skipping
Seen 3 / 227 pages
skipping
Seen 4 / 227 pages
skipping
Seen 5 / 227 pages
skipping
Seen 6 / 227 pages
skipping
Seen 7 / 227 pages
skipping
Seen 8 / 227 pages
skipping
Seen 9 / 227 pages
skipping
Seen 10 / 227 pages
skipping
Seen 11 / 227 pages
skipping
Seen 12 / 227 pages
skipping
Seen 13 / 227 pages
skipping
Seen 14 / 227 pages
skipping
Seen 15 / 227 pages
skipping
Seen 16 / 227 pages
skipping
Seen 17 / 227 pages
skipping
Seen 18 / 227 pages
skipping
Seen 19 / 227 pages
skipping
Seen 20 / 227 pages
skipping
Seen 21 / 227 pages
skipping
Seen 22 / 227 pages
skipping
Seen 23 / 227 pages
skipping
Seen 24 / 227 pages
skipping
Seen 25 / 227 pages
skipping
Seen 26 / 227 pages
skipping
Seen 27 / 227 pages
skipping
Seen 28 / 227 pages
skipping
Seen 29 / 227 pages
skipping
Seen 30 / 227 pages
skipping
Seen 31 / 227 pages
skipping
Seen 32 / 227 pages
skipping
Seen 33 / 227 pages
skipping
Seen 34 / 227 pages
skipping
Seen 35 / 227 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/div[3]/section/ul/li[3]/a"}
  (Session info: chrome=102.0.5005.63)
Stacktrace:
Backtrace:
	Ordinal0 [0x0123B8F3+2406643]
	Ordinal0 [0x011CAF31+1945393]
	Ordinal0 [0x010BC748+837448]
	Ordinal0 [0x010E92E0+1020640]
	Ordinal0 [0x010E957B+1021307]
	Ordinal0 [0x01116372+1205106]
	Ordinal0 [0x011042C4+1131204]
	Ordinal0 [0x01114682+1197698]
	Ordinal0 [0x01104096+1130646]
	Ordinal0 [0x010DE636+976438]
	Ordinal0 [0x010DF546+980294]
	GetHandleVerifier [0x014A9612+2498066]
	GetHandleVerifier [0x0149C920+2445600]
	GetHandleVerifier [0x012D4F2A+579370]
	GetHandleVerifier [0x012D3D36+574774]
	Ordinal0 [0x011D1C0B+1973259]
	Ordinal0 [0x011D6688+1992328]
	Ordinal0 [0x011D6775+1992565]
	Ordinal0 [0x011DF8D1+2029777]
	BaseThreadInitThunk [0x77046739+25]
	RtlGetFullPathName_UEx [0x77548FEF+1215]
	RtlGetFullPathName_UEx [0x77548